In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [1]:
import math
import numpy as np
import paddle.optimizer as opt
import paddle
import paddle.nn as nn

from paddle.nn.initializer import TruncatedNormal, Constant, Assign
trunc_normal_ = TruncatedNormal(std=.02)
zeros_ = Constant(value=0.)
ones_ = Constant(value=1.)
class Identity(nn.Layer):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, input):
        return input


class Mlp(nn.Layer):
    def __init__(self,
                 in_features,
                 hidden_features=None,
                 out_features=None,
                 act_layer=nn.GELU,
                 drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x
def drop_path(x, drop_prob=0., training=False):
    """Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ...
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = paddle.to_tensor(1 - drop_prob)
    shape = (paddle.shape(x)[0], ) + (1, ) * (x.ndim - 1)
    random_tensor = keep_prob + paddle.rand(shape, dtype=x.dtype)
    random_tensor = paddle.floor(random_tensor)  # binarize
    output = x.divide(keep_prob) * random_tensor
    return output


class DropPath(nn.Layer):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """

    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)

class Attention(nn.Layer):
    def __init__(self,
                 dim,
                 num_heads=8,
                 qkv_bias=False,
                 qk_scale=None,
                 attn_drop=0.,
                 proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim**-0.5

        self.qkv = nn.Linear(dim, dim * 3, bias_attr=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        # B= paddle.shape(x)[0]
        N, C = x.shape[1:]
        qkv = self.qkv(x).reshape((-1, N, 3, self.num_heads, C //
                                   self.num_heads)).transpose((2, 0, 3, 1, 4))
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q.matmul(k.transpose((0, 1, 3, 2)))) * self.scale
        attn = nn.functional.softmax(attn, axis=-1)
        attn = self.attn_drop(attn)

        x = (attn.matmul(v)).transpose((0, 2, 1, 3)).reshape((-1, N, C))
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

class Block(nn.Layer):
    def __init__(self,
                 dim,
                 num_heads,
                 mlp_ratio=4.,
                 qkv_bias=False,
                 qk_scale=None,
                 drop=0.,
                 attn_drop=0.,
                 drop_path=0.,
                 act_layer=nn.GELU,
                 norm_layer='nn.LayerNorm',
                 epsilon=1e-5):
        super().__init__()
        self.norm1 = eval(norm_layer)(dim, epsilon=epsilon)
        self.attn = Attention(
            dim,
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            qk_scale=qk_scale,
            attn_drop=attn_drop,
            proj_drop=drop)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path) if drop_path > 0. else Identity()
        self.norm2 = eval(norm_layer)(dim, epsilon=epsilon)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim,
                       hidden_features=mlp_hidden_dim,
                       act_layer=act_layer,
                       drop=drop)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

class Transformer(nn.Layer):
    def __init__(self, base_dim, depth, heads, mlp_ratio,
                 drop_rate=.0, attn_drop_rate=.0, drop_path_prob=None):
        super(Transformer, self).__init__()
        self.layers = nn.LayerList([])
        embed_dim = base_dim * heads

        if drop_path_prob is None:
            drop_path_prob = [0.0 for _ in range(depth)]

        self.blocks = nn.LayerList([
            Block(
                dim=embed_dim,
                num_heads=heads,
                mlp_ratio=mlp_ratio,
                qkv_bias=True,
                drop=drop_rate,
                attn_drop=attn_drop_rate,
                drop_path=drop_path_prob[i],
                norm_layer='nn.LayerNorm',
                epsilon=1e-6,
            )
            for i in range(depth)])

    def forward(self, x):
        n, c, h, w = x.shape
        x = x.transpose((0, 2, 3, 1))
        x = paddle.flatten(x, start_axis=1, stop_axis=2)

        for blk in self.blocks:
            x = blk(x)

        x = x.transpose((0, 2, 1))
        x = x.reshape((n, c, h, w))
        return x

class conv_head_pooling(nn.Layer):
    def __init__(self, in_feature, out_feature, stride,
                 padding_mode='zeros'):
        super(conv_head_pooling, self).__init__()

        self.conv = nn.Conv2D(in_feature, out_feature, kernel_size=stride + 1,
                              padding=stride // 2, stride=stride,
                              padding_mode=padding_mode, groups=in_feature)
        self.fc = nn.Linear(in_feature, out_feature)

    def forward(self, x):

        x = self.conv(x)

        return x


class conv_embedding(nn.Layer):
    def __init__(self, in_channels, out_channels, patch_size,
                 stride, padding):
        super(conv_embedding, self).__init__()
        self.conv = nn.Conv2D(in_channels, out_channels, kernel_size=patch_size,
                              stride=stride, padding=padding, bias_attr=True)

    def forward(self, x):
        x = self.conv(x)
        return x

In [4]:
class PoolingTransformer(nn.Layer):
    def __init__(self, image_size, patch_size, stride, base_dims, depth, heads,
                 mlp_ratio, in_chans=3, attn_drop_rate=.0, drop_rate=.0,
                 drop_path_rate=.0, class_dim=10):
        super(PoolingTransformer, self).__init__()
        self.pool1=paddle.nn.MaxPool2D([7,1],1)
        self.pool2=paddle.nn.AvgPool2D([1,7],1)
        self.pool3=paddle.nn.MaxPool2D([1,7],1)
        self.pool4=paddle.nn.AvgPool2D([7,1],1)
        self.pool5=paddle.nn.AvgPool2D([7,1],1)
        self.pool6=paddle.nn.MaxPool2D([1,7],1)
        self.pool7=paddle.nn.AvgPool2D([1,7],1)
        self.pool8=paddle.nn.MaxPool2D([7,1],1)
        total_block = sum(depth)
        padding = 0
        block_idx = 0

        width = math.floor(
            (image_size + 2 * padding - patch_size) / stride + 1)

        self.base_dims = base_dims
        self.heads = heads
        self.class_dim = class_dim

        self.patch_size = patch_size

        self.pos_embed = self.create_parameter(
            shape=(1, base_dims[0] * heads[0], width, width),
            default_initializer=Assign(
                paddle.randn((1, base_dims[0] * heads[0], width, width))
            ))
        self.add_parameter("pos_embed", self.pos_embed)

        self.patch_embed = conv_embedding(in_chans, base_dims[0] * heads[0],
                                          patch_size, stride, padding)

        self.cls_token = self.create_parameter(
            shape=(1, 1, base_dims[0] * heads[0]),
            default_initializer=Assign(
                paddle.randn((1, 1, base_dims[0] * heads[0]))
            ))
        self.add_parameter("cls_token", self.cls_token)

        self.pos_drop = nn.Dropout(p=drop_rate)

        self.transformers = nn.LayerList([])
        self.pools = nn.LayerList([])

        for stage in range(len(depth)):
            drop_path_prob = [drop_path_rate * i / total_block
                              for i in range(block_idx, block_idx + depth[stage])]
            block_idx += depth[stage]

            self.transformers.append(
                Transformer(base_dims[stage], depth[stage], heads[stage],
                            mlp_ratio, drop_rate, attn_drop_rate, drop_path_prob)
            )
            if stage < len(heads) - 1:
                self.pools.append(
                    conv_head_pooling(base_dims[stage] * heads[stage],
                                      base_dims[stage + 1] * heads[stage + 1],
                                      stride=2
                                      )
                )

        self.norm = nn.LayerNorm(base_dims[-1] * heads[-1]*4, epsilon=1e-6)
        self.embed_dim = base_dims[-1] * heads[-1]

        # Classifier head
        if class_dim > 0:
            self.head1 = nn.Linear(base_dims[-1] * heads[-1]*4, class_dim)
        trunc_normal_(self.pos_embed)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.LayerNorm):
            zeros_(m.bias)
            ones_(m.weight)

    def forward_features(self, x):
        x = self.patch_embed(x)

        pos_embed = self.pos_embed
        x = self.pos_drop(x + pos_embed)

        for stage in range(len(self.pools)):
            x = self.transformers[stage](x)
            x = self.pools[stage](x)
        x = self.transformers[len(
            self.transformers)-1](x)
        x1=self.pool2(self.pool1(x)).flatten(1)
        x2=self.pool4(self.pool3(x)).flatten(1)
        x3=self.pool6(self.pool5(x)).flatten(1)
        x4=self.pool8(self.pool7(x)).flatten(1)
        x=paddle.concat(x=[x1, x2,x3,x4], axis=1, name=None)
        x = self.norm(x)
        return x

    def forward(self, x):
        x = self.forward_features(x)

        if self.class_dim > 0:
            x = self.head1(x)

        return x

def pit_s(**kwargs):
    model = PoolingTransformer(
        image_size=224,
        patch_size=16,
        stride=8,
        base_dims=[48, 48, 48],
        depth=[2, 6, 4],
        heads=[3, 6, 12],
        mlp_ratio=4,
        **kwargs
    )
    return model

model = pit_s()
out = model(paddle.randn((1, 3, 224, 224)))
print(out.shape)

[16, 2304]
[16, 10]


In [6]:
model = pit_s()

paddle.summary(model, (16, 3, 224, 224))

out = model(paddle.randn((16, 3, 224, 224)))

print(out.shape)

[16, 2304]
-------------------------------------------------------------------------------
   Layer (type)         Input Shape          Output Shape         Param #    
     Conv2D-10      [[16, 3, 224, 224]]   [16, 144, 27, 27]       110,736    
 conv_embedding-4   [[16, 3, 224, 224]]   [16, 144, 27, 27]          0       
    Dropout-112     [[16, 144, 27, 27]]   [16, 144, 27, 27]          0       
   LayerNorm-76       [[16, 729, 144]]      [16, 729, 144]          288      
    Linear-154        [[16, 729, 144]]      [16, 729, 432]        62,640     
    Dropout-113     [[16, 3, 729, 729]]   [16, 3, 729, 729]          0       
    Linear-155        [[16, 729, 144]]      [16, 729, 144]        20,880     
    Dropout-114       [[16, 729, 144]]      [16, 729, 144]           0       
   Attention-37       [[16, 729, 144]]      [16, 729, 144]           0       
    Identity-37       [[16, 729, 144]]      [16, 729, 144]           0       
   LayerNorm-77       [[16, 729, 144]]      [16, 72

In [3]:
model = pit_s()
out = model(paddle.randn((1, 3, 224, 224)))
print(out.shape)

[1, 10]


In [4]:
import paddle.vision.transforms as T
from paddle.vision.datasets import Cifar10

paddle.set_device('gpu')

#数据准备
transform = T.Compose([
    T.Resize(size=(224,224)),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],data_format='HWC'),
    T.ToTensor()
])

train_dataset = Cifar10(mode='train', transform=transform)
val_dataset = Cifar10(mode='test',  transform=transform)

Cache file /home/aistudio/.cache/paddle/dataset/cifar/cifar-10-python.tar.gz not found, downloading https://dataset.bj.bcebos.com/cifar/cifar-10-python.tar.gz 
Begin to download

Download finished


In [5]:
class SaveBestModel(paddle.callbacks.Callback):
    def __init__(self, target=0.5, path='./best_model', verbose=0):
        self.target = target
        self.epoch = None
        self.path = path

    def on_epoch_end(self, epoch, logs=None):
        self.epoch = epoch

    def on_eval_end(self, logs=None):
        if logs.get('acc_top1') > self.target:
            self.target = logs.get('acc_top1')
            self.model.save(self.path)
            print('best acc_top1 is {} at epoch {}'.format(self.target, self.epoch+1))
callback_savebestmodel = SaveBestModel(target=0.5, path='./best_model')
model = pit_s()
model = paddle.Model(model)

In [6]:
scheduler = opt.lr.CosineAnnealingDecay(learning_rate=0.000375, T_max=50)
model.prepare(optimizer=paddle.optimizer.AdamW(weight_decay=0.05,learning_rate=scheduler, parameters=model.parameters()),
              loss=paddle.nn.CrossEntropyLoss(),
              metrics=paddle.metric.Accuracy(topk=(1,2,3,4,5)))

visualdl=paddle.callbacks.VisualDL(log_dir='visual_log') # 开启训练可视化
callbacks=[visualdl,callback_savebestmodel]
model.fit(
    train_data=train_dataset, 
    eval_data=val_dataset, 
    batch_size=16, 
    epochs=50,
    verbose=1,
    save_dir='checkpoint',
    save_freq=10,
    callbacks=callbacks
)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/50


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 3125/3125 [==============================] - loss: 0.9127 - acc_top1: 0.4418 - acc_top2: 0.6453 - acc_top3: 0.7630 - acc_top4: 0.8376 - acc_top5: 0.8905 - 87ms/step         
save checkpoint at /home/aistudio/checkpoint/0
Eval begin...
step 625/625 [==============================] - loss: 1.7668 - acc_top1: 0.5390 - acc_top2: 0.7397 - acc_top3: 0.8415 - acc_top4: 0.9024 - acc_top5: 0.9407 - 36ms/step         
Eval samples: 10000
best acc_top1 is 0.539 at epoch 1
Epoch 2/50
step 3125/3125 [==============================] - loss: 1.3565 - acc_top1: 0.5918 - acc_top2: 0.7879 - acc_top3: 0.8751 - acc_top4: 0.9248 - acc_top5: 0.9554 - 87ms/step         
Eval begin...
step 625/625 [==============================] - loss: 1.6867 - acc_top1: 0.6168 - acc_top2: 0.8038 - acc_top3: 0.8912 - acc_top4: 0.9331 - acc_top5: 0.9627 - 35ms/step         
Eval samples: 10000
best acc_top1 is 0.6168 at epoch 2
Epoch 3/50
step 3125/3125 [==============================] - loss: 0.5813 - acc_top1: 0.6566 